# Flask蓝图

### 创建蓝图对象

```python
# admin.py文件

#Blueprint必须指定两个参数，admin表示蓝图的名称，__name__表示蓝图所在模块
admin = Blueprint('admin',__name__)
```

### 注册蓝图路由

```python
# admin.py文件

@admin.route('/')
def admin_index():
    return 'admin_index'
```

### 注册蓝图

```python
# main.py文件

app.register_blueprint(admin,url_prefix='/admin')
```

# Flask大型项目的目录结构

- 大型项目模板：
https://github.com/Wangler2333/flask_project_demo

```
../flask_project/
├── __init__.py
├── app
│   ├── __init__.py
│   ├── email.py
│   ├── main
│   │   ├── __init__.py
│   │   ├── errors.py
│   │   ├── forms.py
│   │   ├── views.py
│   ├── models.py
│   ├── static
│   └── templates
├── config.py
├── manage.py
└── test
    └── __init__.py
```

### app/__init__.py

- 构建app_create函数用来进行配置初始化，动态加载配置文件
- 使用工厂模式构建app

```python
#!/usr/bin/python3 
# -*-coding:utf-8-*- 
# @Author: 李多
# @Time: 2018年05月08日14时04分 
# 说明: 
# 总结:

from flask import Flask
from flask.ext.bootstrap import Bootstrap
from flask.ext.mail import Mail
from flask.ext.moment import Moment
from flask.ext.sqlalchemy import SQLAlchemy
from config import config

bootstrap = Bootstrap()
mail = Mail()
moment = Moment()
db = SQLAlchemy()


def app_create(config_name):
    app = Flask(__name__)
    app.config.from_object(config[config_name])
    config[config_name].init_app(app)

    bootstrap.init_app(app)
    mail.init_app(app)
    moment.init_app(app)
    db.init_app(app)
    # 路由和其他处理程序定义
    # ...
    from .main import main as main_blueprint
    app.register_blueprint(main_blueprint)
    return app
```

### app/main/__init__.py

- 构建main中views使用的蓝图

```python
#!/usr/bin/python3 
# -*-coding:utf-8-*- 
# @Author: 李多
# @Time: 2018年05月08日14时07分 
# 说明: 
# 总结:

from flask import Blueprint

main = Blueprint('main', __name__)
from . import views, errors
```

### config.py

- 标准配置文件

```python
#!/usr/bin/python3 
# -*-coding:utf-8-*- 
# @Author: 李多
# @Time: 2018年05月08日14时05分 
# 说明: 
# 总结:

import os

basedir = os.path.abspath(os.path.dirname(__file__))


class Config:
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'hard to guess string'
    SQLALCHEMY_COMMIT_ON_TEARDOWN = True
    FLASK_MAIL_SUBJECT_PREFIX = '[Flask]'
    FLASK_MAIL_SENDER = 'Flask Admin <flasky@example.com>'
    FLASK_ADMIN = os.environ.get('FLASK_ADMIN')

    def __init__(self):
        pass

    @staticmethod
    def init_app(app):  # 执行当前需要的环境的初始化

        pass


class DevelopmentConfig(Config):  # 开发环境
    DEBUG = True
    MAIL_SERVER = 'smtp.googlemail.com'
    MAIL_PORT = 587
    MAIL_USE_TLS = True
    MAIL_USERNAME = os.environ.get('MAIL_USERNAME')
    MAIL_PASSWORD = os.environ.get('MAIL_PASSWORD')
    SQLALCHEMY_DATABASE_URI = os.environ.get('DEV_DATABASE_URL') or 'sqlite:///' + os.path.join(basedir,
                                                                                                'data-dev.sqlite')


class TestingConfig(Config):  # 测试环境
    TESTING = True
    SQLALCHEMY_DATABASE_URI = os.environ.get('TEST_DATABASE_URL') or 'sqlite:///' + os.path.join(basedir,
                                                                                                 'data-test.sqlite')


class ProductionConfig(Config):  # 生产环境
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL') or 'sqlite:///' + os.path.join(basedir, 'data.sqlite')


config = {
    'development': DevelopmentConfig,
    'testing': TestingConfig,
    'production': ProductionConfig,
    'default': DevelopmentConfig
}
```

### manage.py

- 标准启动文件

```python
#!/usr/bin/python3 
# -*-coding:utf-8-*- 
# @Author: 李多
# @Time: 2018年05月08日14时06分 
# 说明: 
# 总结:

import os
from app import app_create, db
from app.models import User, Role
from flask.ext.script import Manager, Shell
from flask.ext.migrate import Migrate, MigrateCommand

app = app_create(os.getenv('FLASK_CONFIG') or 'default')
manager = Manager(app)
migrate = Migrate(app, db)


def make_shell_context():
    return dict(app=app, db=db, User=User, Role=Role)


manager.add_command("shell", Shell(make_context=make_shell_context))
manager.add_command('db', MigrateCommand)

if __name__ == '__main__':
    manager.run()
```

# 单元测试
### assert断言

- assert 表达式：表达式成立为真，继续进行；表达式不成立，抛出异常
- 例如：assert isinstance(num1, int)：判断是否为整数

|常用的断言方法：||
|---|---|
|assertEqual|如果两个值相等，则pass|
|assertNotEqual|如果两个值不相等，则pass|
|assertTrue|判断bool值为True，则pass|
|assertFalse|判断bool值为False，则pass|
|assertIsNone|不存在，则pass|
|assertIsNotNone|存在，则pass|

```python
def fibo(x):
    if x == 0:
        resp = 0
    elif x == 1:
        resp = 1
    else:
        return fibo(x-1) + fibo(x-2)
    return resp
assert fibo(5) == 5
```

### 单元测试

```python
import unittest
class TestClass(unittest.TestCase):

    #该方法会首先执行，相当于做测试前的准备工作
    def setUp(self):
        pass

    #该方法会在测试代码执行完后执行，相当于做测试后的扫尾工作
    def tearDown(self):
        pass
    #测试代码
    def test_app_exists(self):
        pass
```

### Flask单元测试

```python
#!/usr/bin/python3 
# -*-coding:utf-8-*- 
# @Author: 李多
# @Time: 2018年05月08日16时39分 
# 说明: 
# 总结:
import unittest
from Flask_project import app


class TestCase(unittest.TestCase):
    # 创建测试环境，在测试代码执行前执行
    def setUp(self):
        # 获取测试用的app对象
        self.app = app
        # 激活测试标志
        app.config['TESTING'] = True
        # 创建app对象的测试客户端
        self.client = self.app.test_client()

    # 在测试代码执行完成后执行
    def tearDown(self):
        pass

    # 测试代码
    # 测试代码必须以test开头
    def test_email(self):
        # 使用客户端发送测试数据
        # post("/", data={})
        # get("/")
        # 返回对象
        resp = self.client.get('/')
        # 获取返回的数据，返回的是视图return的数据
        print resp.data
        # 使用断言判断返回值
        self.assertEqual(resp.data, 'Sent　Succeed')
            
    #测试数据库模型类
    def test_append_data(self):
        au = Author(name='itcast')
        bk = Book(info='python')
        db.session.add_all([au,bk])
        db.session.commit()
        author = Author.query.filter_by(name='itcast').first()
        book = Book.query.filter_by(info='python').first()
        #断言数据存在
        self.assertIsNotNone(author)
        self.assertIsNotNone(book)


if __name__ == "__main__":
    # 运行所有测试代码
    unittest.main()
```

# Flask部署

- nginx + gunicorn + flask

- pip install gunicorn

- gunicorn -w 4 -b 127.0.0.1:5001 运行文件名称:Flask程序实例名

- sudo apt-get install nginx

```bash
#启动
sudo sbin/nginx
#查看
ps aux | grep nginx
#停止
sudo sbin/nginx -s stop
```

- /usr/local/nginx/conf/nginx.conf

```bash


# 设置负载均衡ip
upstream flask {
    server 127.0.0.1:5000
    server 127.0.0.1:5001
}


server {
    # 监听80端口
    listen 80;
    # 本机
    server_name localhost; 
    # 默认请求的url
    location / {
        # 请求转发到gunicorn服务器
        proxy_pass http://flask; 
        # 设置请求头，并将头信息传递给服务器端 
        proxy_set_header Host $host;
        # 设置转发IP地址
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```